# Experiment #1: Open vSwitch 4 Networks

In this experiment we try to connect 4 networks with different hosts together with a bridge in the middle. We will be using Open vSwitch for the bridge, alternate solutions are Linux Bridge and Behavioral Model (BMv2).

## References
- [Open vSwitch Example](https://github.com/fabric-testbed/jupyter-examples/blob/main/fabric_examples/complex_recipes/openvswitch/openvswitch.ipynb)
- [BMv4](https://github.com/p4lang/behavioral-model)

## TODO's

- Add Open vSwitch flow configuratation on ports and so on.
- Add tools on how to track packets in the bridge.
- measure of congestion.


## Topology


<img src="./fig/Testbed-openvswitch-1.png" width=40%>

## Code

In [2]:
from ipaddress import ip_address, IPv4Address, IPv4Network
import ipaddress
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager


fablib = fablib_manager()
fablib.show_config();

Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Artifact Manager,artifacts.fabric-testbed.net
Token File,/home/alphani/.fabric_token.json
Project ID,220c2d8a-ae05-465a-a98b-8178415e1b30
Bastion Host,bastion.fabric-testbed.net
Bastion Username,sa2994_0000240697
Bastion Private Key File,/home/alphani/work/fabric_config/fabric_bastion_key
Slice Public Key File,/home/alphani/work/fabric_config/slice_key.pub
Slice Private Key File,/home/alphani/work/fabric_config/slice_key


In [3]:
slice_name= "Testbed-openvswitch-1"

sites = [site1] = fablib.get_random_sites(count=1)
print(f"Sites: {sites}")

site_node_count = 4
bridge1_name = 'bridge1'

Sites: ['UTAH']


In [4]:
slice = fablib.new_slice(name=slice_name)
default_image= 'default_ubuntu_22'

In [5]:
bridge1 = slice.add_node(name=bridge1_name, site=site1, cores=2, ram=8, disk=10, image=default_image, host=f"{site1.lower()}-w1.fabric-testbed.net")
bridge1_nic1 = bridge1.add_component(model='NIC_Basic', name='nic_local_1')
bridge1_nic2 = bridge1.add_component(model='NIC_Basic', name='nic_local_2')
bridge1_nic3 = bridge1.add_component(model='NIC_Basic', name='nic_local_3')
bridge1_nic4 = bridge1.add_component(model='NIC_Basic', name='nic_local_4')

In [6]:
i = 0
print(f"Adding nodes to {site1}")
for node_num in range(site_node_count):
    node_name = f"{site1.lower()}{node_num+1}"
    
    node = slice.add_node(name=node_name, site=site1, cores=2, ram=8, disk=10, image=default_image, host=f"{site1.lower()}-w2.fabric-testbed.net")
    iface = node.add_component(model='NIC_Basic', name='nic_local').get_interfaces()[0]    
    net = slice.add_l2network(name=f"net{node_num}")

    net.add_interface(iface)
    net.add_interface(bridge1.get_interfaces()[i])
    i += 1

Adding nodes to UTAH


In [7]:
slice.submit();


Retry: 8, Time: 266 sec


ID,b1b4ad22-1dc8-449e-bc4a-28ecd2f875ed
Name,Testbed-openvswitch-1
Lease Expiration (UTC),2025-01-06 21:24:08 +0000
Lease Start (UTC),2025-01-05 21:24:08 +0000
Project ID,220c2d8a-ae05-465a-a98b-8178415e1b30
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
76e83f42-9e88-47c5-ab1d-eb6eeb044af4,bridge1,2,8,10,default_ubuntu_22,qcow2,utah-w1.fabric-testbed.net,UTAH,ubuntu,2001:1948:417:7:f816:3eff:fe9b:8bea,Active,,ssh -i /home/alphani/work/fabric_config/slice_key -F /home/alphani/work/fabric_config/ssh_config ubuntu@2001:1948:417:7:f816:3eff:fe9b:8bea,/home/alphani/work/fabric_config/slice_key.pub,/home/alphani/work/fabric_config/slice_key
6654dd31-71c0-42c6-b4a8-16923db4fde9,utah1,2,8,10,default_ubuntu_22,qcow2,utah-w2.fabric-testbed.net,UTAH,ubuntu,2001:1948:417:7:f816:3eff:feaf:9eed,Active,,ssh -i /home/alphani/work/fabric_config/slice_key -F /home/alphani/work/fabric_config/ssh_config ubuntu@2001:1948:417:7:f816:3eff:feaf:9eed,/home/alphani/work/fabric_config/slice_key.pub,/home/alphani/work/fabric_config/slice_key
51a64a6b-3f3c-4bc6-8f50-a3a1f09fe3e0,utah2,2,8,10,default_ubuntu_22,qcow2,utah-w2.fabric-testbed.net,UTAH,ubuntu,2001:1948:417:7:f816:3eff:fedd:317b,Active,,ssh -i /home/alphani/work/fabric_config/slice_key -F /home/alphani/work/fabric_config/ssh_config ubuntu@2001:1948:417:7:f816:3eff:fedd:317b,/home/alphani/work/fabric_config/slice_key.pub,/home/alphani/work/fabric_config/slice_key
afc49edf-4e94-425c-b205-5ad89b924865,utah3,2,8,10,default_ubuntu_22,qcow2,utah-w2.fabric-testbed.net,UTAH,ubuntu,2001:1948:417:7:f816:3eff:fe22:ba2,Active,,ssh -i /home/alphani/work/fabric_config/slice_key -F /home/alphani/work/fabric_config/ssh_config ubuntu@2001:1948:417:7:f816:3eff:fe22:ba2,/home/alphani/work/fabric_config/slice_key.pub,/home/alphani/work/fabric_config/slice_key
a079afcf-1a55-4909-aba8-39dffba8b533,utah4,2,8,10,default_ubuntu_22,qcow2,utah-w2.fabric-testbed.net,UTAH,ubuntu,2001:1948:417:7:f816:3eff:fe46:fe41,Active,,ssh -i /home/alphani/work/fabric_config/slice_key -F /home/alphani/work/fabric_config/ssh_config ubuntu@2001:1948:417:7:f816:3eff:fe46:fe41,/home/alphani/work/fabric_config/slice_key.pub,/home/alphani/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
701ab625-655f-45c2-bc0f-35a04cc2b6fc,net0,L2,L2Bridge,UTAH,None,None,Active,
940088fa-4817-4084-b8d4-86075f5dc075,net1,L2,L2Bridge,UTAH,None,None,Active,
1d0716f7-b3e5-4e0e-b0fa-b7caf5b6eeb6,net2,L2,L2Bridge,UTAH,None,None,Active,
6b16b8fd-68a0-4bc0-bfe6-db1c82c796c4,net3,L2,L2Bridge,UTAH,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
bridge1-nic_local_4-p1,p1,bridge1,net2,100,config,,0E:2F:49:4C:B6:5E,enp8s0,enp8s0,fe80::c2f:49ff:fe4c:b65e,6,HundredGigE0/0/0/5
bridge1-nic_local_2-p1,p1,bridge1,net3,100,config,,0E:C6:90:95:60:FC,enp9s0,enp9s0,fe80::cc6:90ff:fe95:60fc,6,HundredGigE0/0/0/5
bridge1-nic_local_3-p1,p1,bridge1,net0,100,config,,0E:2B:D9:A9:F5:84,enp7s0,enp7s0,fe80::c2b:d9ff:fea9:f584,6,HundredGigE0/0/0/5
bridge1-nic_local_1-p1,p1,bridge1,net1,100,config,,12:98:E2:68:0A:58,enp10s0,enp10s0,fe80::1098:e2ff:fe68:a58,6,HundredGigE0/0/0/5
utah1-nic_local-p1,p1,utah1,net0,100,config,,26:58:79:A9:21:35,enp7s0,enp7s0,fe80::2458:79ff:fea9:2135,6,HundredGigE0/0/0/7
utah2-nic_local-p1,p1,utah2,net1,100,config,,2A:68:97:FE:D1:FC,enp7s0,enp7s0,fe80::2868:97ff:fefe:d1fc,6,HundredGigE0/0/0/7
utah3-nic_local-p1,p1,utah3,net2,100,config,,2A:A1:3F:87:17:C4,enp7s0,enp7s0,fe80::28a1:3fff:fe87:17c4,6,HundredGigE0/0/0/7
utah4-nic_local-p1,p1,utah4,net3,100,config,,2A:AE:D5:52:A4:4C,enp7s0,enp7s0,fe80::28ae:d5ff:fe52:a44c,6,HundredGigE0/0/0/7



Time to print interfaces 291 seconds


# OVS 

In [8]:
try:
    for node in slice.get_nodes():
        if node.get_name().startswith("bridge"):
            stdout, stderr = node.execute('yes | sudo apt-get -y update && sudo apt-get upgrade', quiet=True) 
            stdout, stderr = node.execute('yes | sudo apt-get -y install openvswitch-switch openvswitch-common', quiet=True)
            stdout, stderr = node.execute('sudo apt-get -y install net-tools', quiet=True)
            print(f"done bridge: {node.get_name()}")
    print("Done")
except Exception as e:
    print(f"Exception: {e}")

done bridge: bridge1
Done


In [9]:
bridge1 = slice.get_node(name=bridge1_name)
stdout, stderr = bridge1.execute('sudo ovs-vsctl add-br br0')
for interface in bridge1.get_interfaces():
    stdout, stderr = bridge1.execute(f'sudo ovs-vsctl add-port br0 {interface.get_physical_os_interface_name()}')
    #Remove IP addresses for all interfaces
    stdout, stderr = bridge1.execute(f'sudo ifconfig {interface.get_physical_os_interface_name()} 0')
    
#bring the bridge up
stdout, stderr = bridge1.execute('sudo ifconfig br0 up')

print("Done")

Done


In [10]:
stdout, stderr = bridge1.execute('sudo ovs-vsctl set bridge br0 stp_enable=true')

In [11]:
stdout, stderr = bridge1.execute('sudo ovs-appctl stp/show')

---- br0 ----
Root ID:
  stp-priority  32768
  stp-system-id   82:49:ac:6e:5a:42
  stp-hello-time  2s
  stp-max-age     20s
  stp-fwd-delay   15s
  This bridge is the root

Bridge ID:
  stp-priority  32768
  stp-system-id   82:49:ac:6e:5a:42
  stp-hello-time  2s
  stp-max-age     20s
  stp-fwd-delay   15s

  Interface  Role       State      Cost  Pri.Nbr 
  ---------- ---------- ---------- ----- -------
  enp7s0     designated listening  2     128.1
  enp8s0     designated listening  2     128.2
  enp10s0    designated listening  2     128.3
  enp9s0     designated listening  2     128.4



# Host Setup 

In [12]:
networks = [
    "10.10.10.1/24",
    "10.10.10.2/24",
    "10.10.10.3/24",
    "10.10.10.4/24"
]
for i in range(4):
    host = slice.get_node(name=f'{site1.lower()}{i+1}')
    stdout, stderr = host.execute('sudo apt-get -y install net-tools', quiet=True)
    stdout, stderr = host.execute(f'sudo ip link set dev {host.get_interfaces()[0].get_physical_os_interface_name()} up', quiet=True)
    stdout, stderr = host.execute(f'sudo ip addr add {networks[i]} dev {host.get_interfaces()[0].get_physical_os_interface_name()}', quiet=True)



In [13]:
host1 = slice.get_node(name=f'{site1.lower()}1')
stdout, stderr = host1.execute('ping 10.10.10.2 -c 5')

PING 10.10.10.2 (10.10.10.2) 56(84) bytes of data.
64 bytes from 10.10.10.2: icmp_seq=1 ttl=64 time=0.581 ms
64 bytes from 10.10.10.2: icmp_seq=2 ttl=64 time=0.135 ms
64 bytes from 10.10.10.2: icmp_seq=3 ttl=64 time=0.137 ms
64 bytes from 10.10.10.2: icmp_seq=4 ttl=64 time=0.148 ms
64 bytes from 10.10.10.2: icmp_seq=5 ttl=64 time=0.146 ms

--- 10.10.10.2 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4082ms
rtt min/avg/max/mdev = 0.135/0.229/0.581/0.175 ms


In [14]:
stdout, stderr = host1.execute('ping 10.10.10.3 -c 5')

PING 10.10.10.3 (10.10.10.3) 56(84) bytes of data.
64 bytes from 10.10.10.3: icmp_seq=1 ttl=64 time=0.552 ms
64 bytes from 10.10.10.3: icmp_seq=2 ttl=64 time=0.173 ms
64 bytes from 10.10.10.3: icmp_seq=3 ttl=64 time=0.141 ms
64 bytes from 10.10.10.3: icmp_seq=4 ttl=64 time=0.145 ms
64 bytes from 10.10.10.3: icmp_seq=5 ttl=64 time=0.130 ms

--- 10.10.10.3 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4079ms
rtt min/avg/max/mdev = 0.130/0.228/0.552/0.162 ms


In [15]:
stdout, stderr = host1.execute('ping 10.10.10.4 -c 5')

PING 10.10.10.4 (10.10.10.4) 56(84) bytes of data.
64 bytes from 10.10.10.4: icmp_seq=1 ttl=64 time=0.448 ms
64 bytes from 10.10.10.4: icmp_seq=2 ttl=64 time=0.160 ms
64 bytes from 10.10.10.4: icmp_seq=3 ttl=64 time=0.211 ms
64 bytes from 10.10.10.4: icmp_seq=4 ttl=64 time=0.157 ms
64 bytes from 10.10.10.4: icmp_seq=5 ttl=64 time=0.192 ms

--- 10.10.10.4 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4103ms
rtt min/avg/max/mdev = 0.157/0.233/0.448/0.109 ms
